In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import folium


   # Creating Classes

   **Road data handling class**

In [2]:
class Road_Features():
    """
    This class reads the csvs of Road data , clean and returen the organized dataFrames
    """
    def __init__(self):
        self.Signal_Handling()
  
        
    
        self.Speed_Handling()
        self.Camera_Handling()
        self.Incidents_Handling()
        self.Sign_Handling()
        self.Volume_Handling()
        

    def Speed_Handling(self):
        self.speed_limit=pd.read_csv('Speed_Limits.csv')
        
        self.speed_limit.rename(columns={'STREET_NAME':'Location','SPEED':'Speed','multiline':'Coordinates','CREATED_DT':'Date'},inplace=True)
        coord=self.speed_limit['Coordinates'].str.split(expand=True)
        
        for i in range(len(coord.columns)):
            if(i>0):
                coord[i]= coord[i].str.strip(')(,')
        coord.drop(columns=0,inplace=True)
        self.speed_limit['Date']=pd.to_datetime(self.speed_limit['Date'])
        self.speed_limit['Year']=pd.DatetimeIndex(self.speed_limit['Date']).year
        self.speed_limit=self.speed_limit[['Year','Speed']]
        self.speed_limit=pd.merge(self.speed_limit,coord,left_index=True,right_index=True)
        self.speed_limit=self.speed_limit.loc[self.speed_limit['Year']<=2018]
    def Camera_Handling(self):
        self.cameras=pd.read_csv('Traffic_Camera_Locations.csv')
        self.cameras=self.cameras[['longitude','latitude']]
#         print(self.cameras)
    def Incidents_Handling(self):
        self.incidents=pd.read_csv('Traffic_Incidents.csv')
        self.incidents.rename(columns={'INCIDENT INFO':'Location','START_DT':'Date'},inplace=True)
        self.incidents['Date']=pd.to_datetime(self.incidents['Date'])
        self.incidents['Year']=pd.DatetimeIndex(self.incidents['Date']).year
        self.incidents=self.incidents[['Location','Year','Longitude','Latitude','Count']]
        self.incidents=self.incidents.loc[self.incidents['Year']==2018]
        self.incidents=self.incidents.groupby('Location').sum().reset_index()
        self.incidents['Longitude']=self.incidents['Longitude']/self.incidents['Count']
        self.incidents['Latitude']=self.incidents['Latitude']/self.incidents['Count']
        self.incidents['Year']=self.incidents['Year']/self.incidents['Count']
    def Signal_Handling(self):
        self.signals=pd.read_csv('Traffic_Signals.csv')
        self.signals.rename(columns={'INSTDATE':'Date','INT_TYPE':'Signal Type','latitude':'Latitude',\
                                     'longitude':'Longitude'},inplace=True)
        self.signals['Intersection']=self.signals['FIRSTROAD']+' & '+self.signals['SECONDROAD']
        self.signals['Date']=pd.to_datetime(self.signals['Date'])
        self.signals['Year']=pd.DatetimeIndex(self.signals['Date']).year
        self.signals['Signal Type']=self.signals['Signal Type'].astype(str)
        self.signals=self.signals[['Intersection','Signal Type','Year','Longitude','Latitude','Count']]
        
        self.signals=self.signals.loc[self.signals['Year']<=2018]
        self.signals=self.signals.groupby('Intersection').sum().reset_index()
        
        self.signals['Longitude']=self.signals['Longitude']/self.signals['Count']
        self.signals['Latitude']=self.signals['Latitude']/self.signals['Count'] 
        self.signals['Year']=self.signals['Year']/self.signals['Count'] 
        
    def Sign_Handling(self):
        self.signs=pd.read_csv('Traffic_Signs.csv')
        self.signs.rename(columns={'BLADE_TYPE':'Type','INSTDATE':'Date'},inplace=True)
        
        self.signs['Date']=pd.to_datetime(self.signs['Date'])
        self.signs['Year']=pd.DatetimeIndex(self.signs['Date']).year
        self.signs['Count']=1
        self.signs=self.signs[['Type','Year','POINT','Count']]
        self.signs=self.signs.groupby('POINT').sum().reset_index()
        self.signs['Year']=self.signs['Year']/self.signs['Count']
        self.signs=self.signs.loc[self.signs['Year']<=2018]
        coord=self.signs['POINT'].str.split(expand=True)
#         
        for i in range(len(coord.columns)):
            
            if (i>0):
                coord[i]= coord[i].str.strip(',()')

        coord.drop(columns=0,inplace=True)
        self.signs.drop(columns='POINT',inplace=True)
        self.signs=pd.merge(self.signs,coord,left_index=True,right_index=True)

    def Volume_Handling(self):
        self.volume=pd.read_csv('Traffic_Volumes_for_2018.csv')
        self.volume.rename(columns={'YEAR':'Year','SECNAME':'Location','VOLUME':'Volume'},inplace=True)
        coord=self.volume['multilinestring'].str.split(expand=True)
        for i in range(len(coord.columns)):
            
            if (i>0):
                coord[i]= coord[i].str.strip(',()')
        coord.drop(columns=0,inplace=True)
        self.volume=self.volume[['Year','Location','Volume']]
        self.volume=pd.merge(self.volume,coord,left_index=True,right_index=True)
    
    def get_Speed(self):
        return self.speed_limit
    def get_Camera(self):
        return self.cameras
    def get_Incident(self):
        return self.incidents
    def get_Signal (self):
        return self.signals
    def get_Sign (self):
        return self.signs
    def get_Volume(self):
        return self.volume
        

**Weather data Class**

In [3]:
class Weather_Features():
#     def __init__(self):
#         self.download_weather_data()
        
    def download_weather_data(self):
        self.url_template = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={station}&Year={year}&Month={month}&Day=14&timeframe={timeframe}&submit=Download+Data"
        self.url = self.url_template.format(station = 50430, year=2018, month=1, timeframe = 1)
        self.weather_data = pd.read_csv(self.url, index_col='Date/Time', parse_dates=True)
        self.weather_data.columns = [col.replace('\xb0', '') for col in self.weather_data.columns]
        self.data_df = pd.DataFrame(self.weather_data)
        self.data_df = self.data_df[['Temp (C)','Visibility (km)']].describe().reset_index()
        return self.data_df


**Map Class**

In [176]:
class Map ():
    """
    this class generates the map with boundary and gridlines, also generate the grid coordinates
    Todo- add markers for each grid to popup stats per grid
    """
    def __init__(self):
        self.boundary=pd.read_csv('City_Boundary_layer.csv')
        self.boundary=self.boundary['the_geom'].str.split(expand=True)
        
        
        for i in range(len(self.boundary.columns)):
            if(i>0):
                self.boundary[i]= self.boundary[i].str.strip(')(,')
        self.boundary.drop(columns=0,inplace=True)
        self.Boundary()
        self.draw_Boundary()
        self.draw_Grid()
        self.generate_Grid_Coord()
        
    def Boundary(self):
#         self.boundary_coordinates=[]
       
        df=pd.DataFrame(columns=['Lat','Long'])
        for i in range(0,len(self.boundary.columns),2):
            if(i+1)<len(self.boundary.columns):
                df = df.append({'Long':self.boundary.iloc[0,(i)],'Lat':self.boundary.iloc[0,(i+1)]}, ignore_index=True)
#                 tes['Long'].append(float(self.boundary.iloc[0,(i)])
#                 tes['Lat'].append(float(self.boundary.iloc[0,(i+1)])
#                 temp=[float(self.boundary.iloc[0,i+1]),float(self.boundary.iloc[0,(i)])]
#                 self.boundary_coordinates.append(temp)
            else:
                break
        self.min_lat=float(df['Lat'].min())
        self.max_lat=float(df['Lat'].max())
        self.min_long=float(df['Long'].min())
        self.max_long=float(df['Long'].max())
#         print(self.min_lat,self.max_long)
#         print(self.min_lat,self.min_long)
#         print(self.max_lat,self.min_long)
#         print(self.max_lat,self.max_long)
    def draw_Boundary(self):
        self.m=folium.Map(location=[51.048615,-114.070847],zoom_start=10)
        cord=[[self.min_lat,self.max_long],[self.min_lat,self.min_long],
              [self.max_lat,self.min_long],[self.max_lat,self.max_long],[self.min_lat,self.max_long]]
        
        f1=folium.FeatureGroup("Boundary")
        line=folium.vector_layers.PolyLine(cord,color='blue').add_to(f1)
#         line=folium.vector_layers.PolyLine(self.boundary_coordinates,popup='<b>boundary</b>',\
#                                      tooltip='boundary',color='blue',weight=10).add_to(f1)
        f1.add_to(self.m)
        
        
        
#         print('hter')
#         display(m)
    def draw_Grid(self):
        f2=folium.FeatureGroup("Grid")
        self.diff_long=(self.max_long-self.min_long)/10
        self.diff_lat=(self.max_lat-self.min_lat)/10
        for i in range(10):
            cord=[[self.min_lat,(self.max_long-self.diff_long*i)],[self.max_lat,(self.max_long-self.diff_long*i)]]
            line=folium.vector_layers.PolyLine(cord,color='blue').add_to(f2)
        for i in range(10):
            cord=[[(self.max_lat-self.diff_lat*i),self.max_long],[(self.max_lat-self.diff_lat*i),self.min_long]]
            line=folium.vector_layers.PolyLine(cord,color='blue').add_to(f2)
        
#         folium.Marker(location=[self.max_lat,self.max_long],popup='max,max',color='red').add_to(self.m)
#         folium.Marker(location=[self.min_lat,self.min_long],popup='min,min',color='red').add_to(self.m)
#         folium.Marker(location=[self.max_lat-self.diff_lat,self.max_long],popup='max-1/10,max',color='red').add_to(self.m)
        f2.add_to(self.m)
        folium.LayerControl().add_to(self.m)
        
    def generate_Grid_Coord(self):
        self.grid_coord=pd.DataFrame(index=['max_lat','min_lat','max_long','min_long'])
        count=0
        key=1
        while(count<=9):
            for i in range(10):
                for k in range(count,(count+1)):
                    max_lat=(self.max_lat-self.diff_lat*k)
                    min_lat=(self.max_lat-self.diff_lat*(k+1))
                    max_long=(self.max_long-self.diff_long*i)
                    min_long=(self.max_long-self.diff_long*(i+1))
#                     a=[(self.max_lat-self.diff_lat*k),(self.max_long-self.diff_long*i)]
#                     b=[(self.max_lat-self.diff_lat*k),(self.max_long-self.diff_long*(i+1))]
#                     c=[(self.max_lat-self.diff_lat*(k+1)),(self.max_long-self.diff_long*(i+1))]
#                     d=[(self.max_lat-self.diff_lat*(k+1)),(self.max_long-self.diff_long*i)]
                    
#                 folium.Marker(location=a,popup='a {}'.format(key),color='red').add_to(self.m)
#                 folium.Marker(location=b,popup='b {}'.format(key),color='red').add_to(self.m)
#                 folium.Marker(location=c,popup='c {}'.format(key),color='red').add_to(self.m)
#                 folium.Marker(location=d,popup='d {}'.format(key),color='red').add_to(self.m)
#                 self.grid_coord[key]=[a,b,c,d]
                self.grid_coord[key]=[max_lat,min_lat,max_long,min_long]
        
        
                key+=1
            count+=1
        self.m.save('index.html')
        
    def get_Grid_Coord(self):
        return self.grid_coord
        

#Todo:
## find a way to find the intersection between the grid and the csv files
## use a marker for each grid  that contain all the stats 

**Data Analysis Class**

In [199]:
class Data_Analysis():
    """
    This class will find if any of the data fall in the grid
    and calculate the stats
    """
    def __init__(self,speed,camera,incident,signal,sign,volume,weather,grid_coord):
        self.speed=speed
        self.camera=camera
        self.incident=incident
        self.signal=signal
        self.sign=sign
        self.volume=volume
        self.weather=weather
        self.grid_coord=grid_coord
        for i in 
        print(self.speed.iloc[0][60])
#         print(self.speed)
#     def speed_Analysis(self):
#         print(self.speed.iloc[0])

**Main Code**

In [5]:
road=Road_Features()
speed=road.get_Speed()
camera=road.get_Camera()
incident=road.get_Incident()
signal=road.get_Signal()
sign=road.get_Sign()
volume=road.get_Volume()
weather=Weather_Features().download_weather_data()


In [177]:
map=Map()

In [179]:
grid_coord=map.get_Grid_Coord()


In [180]:
print(grid_coord.head())

                 1           2           3           4           5    \
max_lat    51.212425   51.212425   51.212425   51.212425   51.212425   
min_lat    51.175465   51.175465   51.175465   51.175465   51.175465   
max_long -114.315796 -114.270207 -114.224618 -114.179029 -114.133440   
min_long -114.270207 -114.224618 -114.179029 -114.133440 -114.087851   

                 6           7           8           9           10   ...  \
max_lat    51.212425   51.212425   51.212425   51.212425   51.212425  ...   
min_lat    51.175465   51.175465   51.175465   51.175465   51.175465  ...   
max_long -114.087851 -114.042261 -113.996672 -113.951083 -113.905494  ...   
min_long -114.042261 -113.996672 -113.951083 -113.905494 -113.859905  ...   

                 91          92          93          94          95   \
max_lat    50.879782   50.879782   50.879782   50.879782   50.879782   
min_lat    50.842822   50.842822   50.842822   50.842822   50.842822   
max_long -114.315796 -114.270207 -114

In [175]:
# print(volume.head())

   Year    Location  Volume                    1                   2  \
0  2018      5AVS12   22000  -114.06036700906716   51.04831941917631   
1  2018  94AVSEWBON    5000  -114.06876825342002   50.96863425573366   
2  2018    12STNE29    5000  -114.03372206187294  51.053232488239935   
3  2018    100AVNE1    2000  -114.01318801277267  51.143369698984344   
4  2018    106AVSE1   14000  -113.98128756128924   50.95784114177629   

                     3                  4                    5  \
0  -114.05790835100508  51.04824965329041                 None   
1   -114.0714654457777  50.96864796962547                 None   
2  -114.03370889695204  51.05088210489753                 None   
3  -114.01185606469929  51.14336996653213  -114.01052411660841   
4  -113.98123806667255  50.95784139004773  -113.98116948007568   

                   6                    7  ...   479   480   481   482   483  \
0               None                 None  ...  None  None  None  None  None   
1         

In [200]:
dataAnalysis=Data_Analysis(speed,camera,incident,signal,sign,volume,weather,grid_coord)

50.994462374778
